# Workflow prototype: EITI API CSV dump to Datasette

Our goal is to prototype a workflow that would allow us to go from the provided CSV, which contains 42K lines of company, agency and project data, to a SQLITE database. The database can then be imported into Datasette to generate a web interface enabling data exploration as well as API endpoints.

Only minimal data cleaning will be included in this prototype workflow, as the aim is to demonstrate the validity of the process.

## Section 1: Importing the CSV

Loading the CSV and giving it a quick look

In [1]:
# we first import pandas and sqlite_utils, which allow us to process the CSV data then import it into the database.
# we also import sqlite3 to test some queries on the created database directly from python

import pandas as pd
import sqlite_utils
import sqlite3

In [20]:
# we load the CSV and display a random sample of the lines for a first visual check

url = "https://raw.githubusercontent.com/clombion/eiti-ose-demo/main/eiti-api-dump.csv"
df = pd.read_csv(url)
df.sample(n=5)

/var/folders/z_/k9wh63yn4bzb0fzdgjkcx6tw0000gn/T/ipykernel_62250/3964467201.py:4: DtypeWarning: Columns (7,11,12,13,14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


,id,label,self,type,identification,sector,commodities,agency_type,company_type,stock_exchange_listing,...,project_affiliated_companies_start,project_commodities,project_status,project_production_volume,project_unit,project_production,project_currency,summary_data.label,summary_data.iso2,summary_data.year
3398,139623,State Property Management Fund,https://eiti.org/api/v2.0/organisation/139623,agency,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kyrgyz Republic,KG,2004.0
13341,162457,Caspi neft JSC,https://eiti.org/api/v2.0/organisation/162457,company,970 140 000 112,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kazakhstan,KZ,2014.0
40466,183273,"Subsoil use special permit No. 2456, dated 27....",https://eiti.org/api/v2.0/organisation/183273,project,NaN,NaN,NaN,NaN,NaN,NaN,...,Ukrgazvydobuvannya JSC,Natural gas,Production,55930000.0,Sm3,NaN,NaN,Ukraine,UA,2020.0
7535,186458,Statoil Oil & Gas Mozambique AS,https://eiti.org/api/v2.0/organisation/186458,company,400155593,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mozambique,MZ,2010.0
6726,158313,Concedo ASA,https://eiti.org/api/v2.0/organisation/158313,company,988217867,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Norway,NO,2008.0


In [3]:
# a second check, this time of the columns and the data types as guessed by pandas

df.dtypes

id                                      int64
label                                  object
self                                   object
type                                   object
identification                         object
sector                                 object
commodities                           float64
agency_type                            object
company_type                           object
stock_exchange_listing                 object
audited_financial_state                object
project_legal_agreement                object
project_affiliated_companies_start     object
project_commodities                    object
project_status                         object
project_production_volume             float64
project_unit                           object
project_production                     object
project_currency                      float64
summary_data.label                     object
summary_data.iso2                      object
summary_data.year                 

## Section 2: separating the different types of data

The CSV contains company, agency and project data. To make the final database more efficient, we need to split them into their own table

In [4]:
# we create three different dataframes, each of them differentiated thanks to the 'type' column.

df_company = df[df['type'] == 'company']
df_agency = df[df['type'] == 'agency']
df_project = df[df['type'] == 'project']

# for each of the created dataframe, we remove any column that is fully blank, allowing us to keep only the relevant columns

df_company = df_company.dropna(how='all', axis=1)
df_agency = df_agency.dropna(how='all', axis=1)
df_project = df_project.dropna(how='all', axis=1)

Let's visualise the three created datasets while adjusting the datatype of the columns

In [5]:
# we adjust the datatype of the company dataset to reflect the actual nature of the values

df_company = df_company.astype(str)
df_company['id'] = pd.to_numeric(df_company['id'], errors='coerce') #to_numeric is used because astype(str) fails if there are null values
df_company['summary_data.year'] = pd.to_numeric(df_company['summary_data.year'], errors='coerce')

# we display both the final list of columns and their datatypes

df_company.dtypes

id                           int64
label                       object
self                        object
type                        object
identification              object
sector                      object
company_type                object
stock_exchange_listing      object
audited_financial_state     object
summary_data.label          object
summary_data.iso2           object
summary_data.year          float64
dtype: object

In [6]:
# we adjust the datatype of the agency dataset to reflect the actual nature of the values

df_agency = df_agency.astype(str)
df_agency['id'] = pd.to_numeric(df_agency['id'], errors='coerce') 
df_agency['summary_data.year'] = pd.to_numeric(df_agency['summary_data.year'], errors='coerce')

# we display both the final list of columns and their datatypes

df_agency.dtypes

id                      int64
label                  object
self                   object
type                   object
identification         object
agency_type            object
summary_data.label     object
summary_data.iso2      object
summary_data.year     float64
dtype: object

In [7]:
# we adjust the datatype of the project dataset to reflect the actual nature of the values

df_project = df_project.astype(str)
df_project['id'] = pd.to_numeric(df_project['id'], errors='coerce')
df_project['summary_data.year'] = pd.to_numeric(df_project['summary_data.year'], errors='coerce')

# we display both the final list of columns and their datatypes

df_project.dtypes

id                                      int64
label                                  object
self                                   object
type                                   object
project_legal_agreement                object
project_affiliated_companies_start     object
project_commodities                    object
project_status                         object
project_production_volume              object
project_unit                           object
project_production                     object
summary_data.label                     object
summary_data.iso2                      object
summary_data.year                     float64
dtype: object

## Section 3: creating the database

We now use sqlite_utils to create our database. 

In [8]:
# create the database

db = sqlite_utils.Database('eiti_data.db')

# insert cleaned data into the SQLite database

db["companies"].insert_all(df_company.to_dict('records'))
db["agencies"].insert_all(df_agency.to_dict('records'))
db["projects"].insert_all(df_project.to_dict('records'))

<Table projects (id, label, self, type, project_legal_agreement, project_affiliated_companies_start, project_commodities, project_status, project_production_volume, project_unit, project_production, summary_data.label, summary_data.iso2, summary_data.year)>

In [9]:
# display schema of the database

for table_name in db.table_names():
    print(f"Schema of {table_name}:")
    print(db[table_name].schema, "\n")

Schema of companies:
CREATE TABLE [companies] (
   [id] INTEGER,
   [label] TEXT,
   [self] TEXT,
   [type] TEXT,
   [identification] TEXT,
   [sector] TEXT,
   [company_type] TEXT,
   [stock_exchange_listing] TEXT,
   [audited_financial_state] TEXT,
   [summary_data.label] TEXT,
   [summary_data.iso2] TEXT,
   [summary_data.year] FLOAT
) 

Schema of agencies:
CREATE TABLE [agencies] (
   [id] INTEGER,
   [label] TEXT,
   [self] TEXT,
   [type] TEXT,
   [identification] TEXT,
   [agency_type] TEXT,
   [summary_data.label] TEXT,
   [summary_data.iso2] TEXT,
   [summary_data.year] FLOAT
) 

Schema of projects:
CREATE TABLE [projects] (
   [id] INTEGER,
   [label] TEXT,
   [self] TEXT,
   [type] TEXT,
   [project_legal_agreement] TEXT,
   [project_affiliated_companies_start] TEXT,
   [project_commodities] TEXT,
   [project_status] TEXT,
   [project_production_volume] TEXT,
   [project_unit] TEXT,
   [project_production] TEXT,
   [summary_data.label] TEXT,
   [summary_data.iso2] TEXT,
   [

## Section 4

We will now test the creation of foreign keys in the 'projects' table based on company names. That requires a more complex process in order to create a junction table to represent the many to many relationships between companies and projects. 

In [10]:
# Create a table 'project_companies'
db["project_companies"].create({"project_id": str, "company_label": str}, pk=("project_id", "company_label"))

<Table project_companies (project_id, company_label)>

In [11]:
# Populate the 'project_companies' table
for index, row in df_project.iterrows():
    # Split the affiliated company names
    affiliated_companies = row['project_affiliated_companies_start'].split(',')
    for company_name in affiliated_companies:
        # Clean up the company name
        company_name = company_name.strip()
        # Try to insert the record, update if it already exists (in our case, it will just keep the values the same)
        db["project_companies"].upsert({
            'project_id': row['id'],
            'company_label': company_name  # Ensure no leading/trailing whitespace
        }, pk=('project_id', 'company_label'))  # composite primary key


In [12]:
# display schema of the database

for table_name in db.table_names():
    print(f"Schema of {table_name}:")
    print(db[table_name].schema, "\n")

Schema of companies:
CREATE TABLE [companies] (
   [id] INTEGER,
   [label] TEXT,
   [self] TEXT,
   [type] TEXT,
   [identification] TEXT,
   [sector] TEXT,
   [company_type] TEXT,
   [stock_exchange_listing] TEXT,
   [audited_financial_state] TEXT,
   [summary_data.label] TEXT,
   [summary_data.iso2] TEXT,
   [summary_data.year] FLOAT
) 

Schema of agencies:
CREATE TABLE [agencies] (
   [id] INTEGER,
   [label] TEXT,
   [self] TEXT,
   [type] TEXT,
   [identification] TEXT,
   [agency_type] TEXT,
   [summary_data.label] TEXT,
   [summary_data.iso2] TEXT,
   [summary_data.year] FLOAT
) 

Schema of projects:
CREATE TABLE [projects] (
   [id] INTEGER,
   [label] TEXT,
   [self] TEXT,
   [type] TEXT,
   [project_legal_agreement] TEXT,
   [project_affiliated_companies_start] TEXT,
   [project_commodities] TEXT,
   [project_status] TEXT,
   [project_production_volume] TEXT,
   [project_unit] TEXT,
   [project_production] TEXT,
   [summary_data.label] TEXT,
   [summary_data.iso2] TEXT,
   [

In [13]:


# Connect to the SQLite database
conn = sqlite3.connect('eiti_data.db')

# Create a cursor object
c = conn.cursor()

# Define the company name. The query is designed to allow substrings, instead of the exact match of the name, allowing subsidiaries to be included.
company_name = "TOTAL"

# Create the SQL query. 
sql = """
SELECT p.label, pc.company_label
FROM projects p
JOIN project_companies pc ON p.id = pc.project_id
WHERE pc.company_label LIKE ?
"""

# Execute the SQL query
c.execute(sql, ('%' + company_name + '%',))

# Fetch all the records
records = c.fetchall()

# Close the connection
conn.close()

In [14]:
# test the result by printing a company and its related projects

data = []

for record in records:
    project_name, company_name = record
    data.append({'Company Name': company_name, 'Project Name': project_name})

df_foreign_key_test = pd.DataFrame(data)
df_foreign_key_test.sample(10)

,Company Name,Project Name
28,TOTALENERGIES E&P UK LIMITED (00811900),TOTALENERGIES E&P UK LIMITED (00811900)
68,Total E&P Nederland,L1d
8,TOTAL AUSTRAL SA,Aries
7,TOTAL AUSTRAL SA,Carina
47,TOTALENERGIES E&P UK LIMITED (00811900),TOTALENERGIES E&P UK LIMITED (00811900)
39,TOTALENERGIES E&P UK LIMITED (00811900),TOTALENERGIES E&P UK LIMITED (00811900)
14,TOTSA TOTAL Halfaya Contract,HALFAYA
83,Total E&P México,CNH‐R01‐L04‐A1.CS/2016
43,TOTALENERGIES E&P UK LIMITED (00811900),TOTALENERGIES E&P UK LIMITED (00811900)
56,Total,CNH-R02-L01-A15.CS/2017


## Run datasette using our database

Datasette can be extensively customised, allowing the host to create custom API endpoints based on specific SQL queries. This means that the user stories that the EITI secretariat have defined for their stakeholders can be translated into specific SQL queries which can then be added to the parameters of datasette in order to set them as custom API endpoints.

This is in addition of the user interface which allows non technical users to navigate, filter and export the data without making use of the API. This interface can be customised to match EITI's branding guideline.

In [16]:
# we start datasette, which opens another page at http://127.0.0.1:8001 

!datasette serve eiti_data.db

INFO:     Started server process [62308]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
INFO:     127.0.0.1:64881 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64881 - "GET /-/static/app.css?d59929 HTTP/1.1" 200 OK
INFO:     127.0.0.1:64881 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     127.0.0.1:64882 - "GET /eiti_data HTTP/1.1" 200 OK
INFO:     127.0.0.1:64882 - "GET /-/static/app.css?d59929 HTTP/1.1" 200 OK
INFO:     127.0.0.1:64882 - "GET /-/static/sql-formatter-2.3.3.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:64884 - "GET /-/static/codemirror-5.57.0.min.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:64885 - "GET /-/static/codemirror-5.57.0-sql.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:64883 - "GET /-/static/codemirror-5.57.0.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:64886 - "GET /-/static/cm-resize-1.0.1.min.js HTTP/1.1" 200 OK
ERROR: conn=<sqlite3.Connection object at 0x10